In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## 1. 서울 + 결측치


## 2. 열제거 (PM10, PM2.5 제외 제거하기)

## 3. average of 11am , 12pm

## 4. 지오코딩

In [2]:
pip install pandas

In [ ]:
import pandas as pd
import os

# 엑셀 파일이 들어있는 폴더 경로
folder_path = './에어코리아'

# 주소 정보를 담고 있는 CSV 파일
address_file = '기상관측데이터_지점정리.csv'
address = pd.read_csv(address_file)

# 결과를 저장할 빈 데이터프레임 생성
all_results = pd.DataFrame()

# 폴더 내의 모든 엑셀 파일 처리
for filename in os.listdir(folder_path):
    if filename.endswith(".xlsx"):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_excel(file_path, engine='openpyxl')

        # 1. 서울 + 결측치
        df_drop = df.dropna()
        df_seoul = df_drop[df_drop['지역'].str.contains('서울')]

        # 2. 열제거 (PM10, PM2.5 제외 제거하기)
        df_seoul = df_seoul.drop(columns=['SO2', 'CO', 'O3', 'NO2'])

        # 3. average of 11am , 12pm
        df_seoul = df_seoul.loc[df_seoul['측정일시'].astype(str).str[-2:].isin(['11', '12']), :]
        df_seoul['측정일시_날짜'] = df_seoul['측정일시'].astype(str).str[:8]

        grouped_df = df_seoul.groupby(['지역', '측정소코드', '측정일시_날짜', '주소'])
        df_average = grouped_df[['PM10', 'PM25']].mean().reset_index()

        # 주소 정보와 결합
        df_geocoding = pd.merge(df_average, address, on='주소', how='left')

        # 처리된 결과를 결과 데이터프레임에 추가
        all_results = pd.concat([all_results, df_geocoding])

# 최종 결과를 CSV 파일로 저장
output_file = '/content/drive/MyDrive/대기오염 Data/New_airkorea/preprocessing/2023_pre.csv'
all_results.to_csv(output_file, index=False)

print(f"최종 결과가 {output_file}에 저장되었습니다.")


<ipython-input-1-a48bd02601df>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seoul['측정일시_날짜'] = df_seoul['측정일시'].astype(str).str[:8]


최종 결과가 /content/drive/MyDrive/대기오염 Data/New_airkorea/preprocessing/2023_pre.csv에 저장되었습니다.


In [ ]:
import os
import zipfile
import pandas as pd

# 1. 압축 파일 경로 및 지역 정보 CSV 로드
project_folder = "./에어코리아"
address_file = "기상관측데이터_지점정리.csv"
output_folder = "./에어코리아"
address = pd.read_csv(address_file)
location_names = address['지점명'].dropna().unique()

# 결과 저장용 데이터프레임 초기화
all_results = pd.DataFrame()

# 1. 연도별 폴더에 있는 엑셀 파일 처리
for year_folder in ['2019', '2020', '2021', '2022', '2023']:
    year_folder_path = os.path.join(project_folder, year_folder)

    if os.path.exists(year_folder_path):  # 폴더가 존재하는지 확인
        print(f"Processing folder: {year_folder_path}")

        # 2. 해당 폴더 내 엑셀 파일 처리
        for filename in os.listdir(year_folder_path):
            if filename.endswith(".xlsx"):
                file_path = os.path.join(year_folder_path, filename)
                df = pd.read_excel(file_path, engine='openpyxl')

                # 데이터 전처리 - 특정 열 삭제 및 지점명 추가
                for location_name in location_names:
                    # 해당 지점명 데이터 필터링
                    df_filtered = df[df['주소'].str.contains(location_name, na=False)]
                    if df_filtered.empty:
                        continue  # 데이터가 없으면 다음으로 넘어감

                    # 새로운 열에 지점명 추가
                    # 수정된 코드
                    df_filtered.loc[:, '지점명'] = location_name


                    # 불필요한 열 제거
                    drop_columns = ['지역', '망', '측정소명', 'SO2', 'CO', 'O3', 'NO2']
                    df_filtered = df_filtered.drop(columns=[col for col in drop_columns if col in df_filtered.columns])

                    # 특정 시간대 데이터 필터링
                    df_filtered = df_filtered.loc[
                        df_filtered['측정일시'].astype(str).str[-2:].isin(['10', '11']), :
                    ]
                    df_filtered['측정일시_날짜'] = df_filtered['측정일시'].astype(str).str[:8]

                    # 평균 계산
                    grouped_df = df_filtered.groupby(
                        ['지점명', '측정소코드', '측정일시_날짜', '주소']
                    )[['PM10', 'PM25']].mean().reset_index()

                    # 결과 데이터프레임에 추가
                    all_results = pd.concat([all_results, grouped_df], ignore_index=True)

# 3. 동일 연도 데이터끼리 파일로 저장
all_results['측정일시_년도'] = all_results['측정일시_날짜'].str[:4]

for year, year_data in all_results.groupby('측정일시_년도'):
    output_file = os.path.join(output_folder, f"{year}_data.csv")
    year_data.to_csv(output_file, index=False, encoding='utf-8-sig')  # UTF-8-SIG로 저장하여 한글 깨짐 방지

print("작업이 완료되었습니다!")


C:\Users\uaua1\AppData\Local\Temp\ipykernel_11056\3060485097.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['지점명'] = location_name
C:\Users\uaua1\AppData\Local\Temp\ipykernel_11056\3060485097.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['지점명'] = location_name
C:\Users\uaua1\AppData\Local\Temp\ipykernel_11056\3060485097.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

작업이 완료되었습니다!


## 5. 센티넬이랑 matching

In [ ]:
# 미리 처리된 데이터프레임 로드
df = pd.read_csv('/content/drive/MyDrive/대기오염 Data/New_airkorea/preprocessing/2023_pre.csv')

# 새로운 specific_dates 리스트
specific_dates = [
    '20230815', '20230909', '20230924', '20231029', '20231113',
    '20231118', '20231123', '20231128', '20231203', '20231208',
    '20231223', '20231228',
]

# 새로운 파일 경로 리스트
filtered_file_paths = [
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20230815.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20230909.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20230924.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231029.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231113.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231118.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231123.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231128.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231203.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231208.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231223.csv',
    '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231228.csv',
]

# Loop through each specific date and corresponding file path
for i, specific_date in enumerate(specific_dates):
    # Filter the dataframe by the specified date (YYYYMMDD)
    filtered_df = df[df['측정일시_날짜'].astype(str).str[:8].isin([specific_date])]

    # Save the filtered dataframe to a new CSV file
    filtered_df.to_csv(filtered_file_paths[i], index=False)

    # Print confirmation
    print(f"Filtered data saved to {filtered_file_paths[i]}")


Filtered data saved to /content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20230815.csv
Filtered data saved to /content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20230909.csv
Filtered data saved to /content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20230924.csv
Filtered data saved to /content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231029.csv
Filtered data saved to /content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231113.csv
Filtered data saved to /content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231118.csv
Filtered data saved to /content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231123.csv
Filtered data saved to /content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231128.csv
Filtered data saved to /content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching/20231203.csv
Filtered data saved to /content/drive/MyDrive/대기오염

## 6. 센티넬이랑 합치기

In [ ]:
import pandas as pd
import os

# 파일 경로 설정
airkorea_folder = '/content/drive/MyDrive/대기오염 Data/New_airkorea/Sentinel-matching'
sentinel_folder = '/content/drive/MyDrive/대기오염 Data/Sentinel 2 Data/band value/2023/'
output_folder = '/content/drive/MyDrive/대기오염 Data/Sentinel+Airkorea'

# 파일 목록 가져오기
airkorea_files = [f for f in os.listdir(airkorea_folder) if f.endswith('.csv')]
sentinel_files = [f for f in os.listdir(sentinel_folder) if f.endswith('.csv')]

# 공통 파일 목록 찾기
common_files = set(airkorea_files) & set(sentinel_files)

# 병합 기준 컬럼 설정
merge_columns = ['Latitude', 'Longitude']

# 파일 처리
for file_name in common_files:
    try:
        # 파일 읽기
        df_airkorea = pd.read_csv(os.path.join(airkorea_folder, file_name), index_col=0)
        df_sentinel = pd.read_csv(os.path.join(sentinel_folder, file_name))

        # 데이터 병합
        df_merged = pd.merge(df_airkorea, df_sentinel[merge_columns + ['geometry', 'AOT','Band1', 'Band2', 'Band3', 'Band4', 'Band5', 'Band6', 'Band7', 'Band11', 'Band12', 'B8A(NIR)', 'SCL', 'TCI', 'WVP']],
                             on=merge_columns, how='left')
        # 데이터 병합
        #df_merged = pd.merge(df_airkorea, df_sentinel[merge_columns + ['geometry', 'AOT','Band2', 'Band3', 'Band4', 'Band5', 'Band6', 'Band7', 'Band11', 'Band12', 'B8A(NIR)', 'SCL', 'TCI', 'WVP']],
        #                     on=merge_columns, how='left')

        # 병합된 데이터프레임을 저장
        df_merged.to_csv(os.path.join(output_folder, file_name), index=False)

        # 병합 결과 출력 (첫 몇 줄)
        print(f"Processed and saved {file_name}")

    except KeyError as e:
        # 오류 발생 시 메시지 출력 후 넘어가기
        print(f"KeyError: {e} in file {file_name}. Skipping this file.")

    except Exception as e:
        # 기타 오류 발생 시 메시지 출력 후 넘어가기
        print(f"Error: {e} in file {file_name}. Skipping this file.")

print("All files processed.")


Processed and saved 20230706.csv
Processed and saved 20231113.csv
Processed and saved 20230427.csv
Processed and saved 20230201.csv
Processed and saved 20231223.csv
Processed and saved 20230303.csv
Processed and saved 20230402.csv
Processed and saved 20231228.csv
Processed and saved 20231208.csv
Processed and saved 20230112.csv
Processed and saved 20230417.csv
Processed and saved 20231118.csv
Processed and saved 20231123.csv
Processed and saved 20230517.csv
Processed and saved 20230701.csv
Processed and saved 20230815.csv
Processed and saved 20230522.csv
Processed and saved 20230412.csv
Processed and saved 20230721.csv
Processed and saved 20230313.csv
Processed and saved 20230226.csv
Processed and saved 20230127.csv
Processed and saved 20230616.csv
Processed and saved 20230407.csv
Processed and saved 20231029.csv
Processed and saved 20230909.csv
Processed and saved 20231128.csv
Processed and saved 20230221.csv
Processed and saved 20230924.csv
Processed and saved 20230102.csv
Processed 